<a href="https://colab.research.google.com/github/SukratiJain/UnwrapAndScrap/blob/main/Google_Images_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Google Search Result Images Scraper and save locally on pc.
##### Technologies Used: Python3, Requests

In [ ]:
import requests
from lxml import html
import time
from parsel import Selector
import os


class Google_Search_Images:
	def __init__(self):
		self.search_keyword = "cat"

		self.headers={'User-Agent':"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.134 Safari/537.36"}
		self.params = {
						"q": self.search_keyword,
						"source": "lnms",
						"tbm": "isch",
					  }
		self.url = "https://www.google.co.in"
		self.search_url = "https://www.google.co.in/search"
		self.count = 0
		directory = os.getcwd()		### Get current directory
		self.path = directory+"\\"+self.search_keyword
		try:
		    os.mkdir(self.path)		### Create a new folder
		except FileExistsError:
		    pass

	def get_Search_Base_Url(self):
		response = requests.get(url=self.search_url, headers=self.headers,params=self.params)
		tree = html.fromstring(response.text)
		images_url_list = tree.xpath('//*[@class="yWs4tf"]//@src')
		for image_url in images_url_list:
			self.save_Image_Locally(image_url)
			
		content = Selector(text=response.text)
		pagination = content.xpath('//*[@class="frGj1b"]//@href')
		if pagination:
			pagination_url = self.url+pagination[0]
			self.search_Image_Pagination(pagination_url)
		
	def search_Image_Pagination(self, pagination_url):
		time.sleep(1)
		print("Images saved:",self.count)

		response = requests.get(url=pagination_url, headers=self.headers)
		tree = html.fromstring(response.text)

		images_url_list = tree.xpath('//*[@class="yWs4tf"]//@src')
		for image_url in images_url_list:
			self.save_Image_Locally(image_url)
	
		pagination = tree.xpath('//*[@class="frGj1b"]//@href')
		if pagination:
			pagination_url = self.url+pagination[-1]
			self.search_Image_Pagination(pagination_url)
		else:
			print("Images downloaded at location: %s" % self.path)

	def save_Image_Locally(self,image_url):
		self.count += 1
		img_response = requests.get(image_url)
		file = open(self.search_keyword+"/image"+str(self.count)+".png","wb")
		file.write(img_response.content)
		file.close()


if __name__ == '__main__':
	obj = Google_Search_Images()
	obj.get_Search_Base_Url()